In [1]:
########################################只用了PIL,############
import pdb 


import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import os
import sys
import cv2
from PIL import Image
from keras.preprocessing.image import *
from keras.models import load_model
import keras.backend as K
from keras.applications.imagenet_utils import preprocess_input

from models import *

from collections import Counter

from __future__ import division



def draw_color(img):   ##6 classes
    #print img
    img_R = np.zeros((img.shape[0],img.shape[1],3),np.uint8)
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
            #RGB
            if(img[i][j] == 1):
                img_R[i][j][0] = 128
                img_R[i][j][1] = 0
                img_R[i][j][2] = 0
            elif(img[i][j] == 2):
                img_R[i][j][0] = 0
                img_R[i][j][1] = 128
                img_R[i][j][2] = 0
            elif(img[i][j] == 3):
                img_R[i][j][0] = 128
                img_R[i][j][1] = 0
                img_R[i][j][2] = 128
            elif(img[i][j] == 4):
                img_R[i][j][0] = 0
                img_R[i][j][1] = 128
                img_R[i][j][2] = 128
            elif(img[i][j] == 5):
                img_R[i][j][0] = 128
                img_R[i][j][1] = 128
                img_R[i][j][2] = 128
            elif(img[i][j] == 6):
                img_R[i][j][0] = 255
                img_R[i][j][1] = 255
                img_R[i][j][2] = 255
            elif(img[i][j] == 255):
                img_R[i][j][0] = 255
                img_R[i][j][1] = 255
                img_R[i][j][2] = 255
            else:
                img_R[i][j][0] = 0
                img_R[i][j][1] = 0
                img_R[i][j][2] = 0
    return img_R


##测试predict,预测每一小块
def small_predict(model, imgTemp):
    
#     img_org = Image.fromarray(imgTemp)
#     w, h = img_org.size
#     img = img_org.resize((320,320))
    imgTemp = np.array(imgTemp, dtype=np.float32)
    
    x = np.expand_dims(imgTemp, axis=0)
    x = preprocess_input(x)
    pred = model.predict(x)
    
    pred = np.argmax(np.squeeze(pred), axis=-1).astype(np.uint8)
    
    return pred


def big_predict(model,src_arr,step_row,step_col,block_row,block_col):
    
    bigPred_R = np.zeros((src_arr.shape[0],src_arr.shape[1]),np.uint8)
    
    ##count=1
    
    for y in range(0,src_arr.shape[0]-block_row+1,step_row*2):  #####step*2
        for x in range(0,src_arr.shape[1]-block_col+1,step_col*2):
            #输出需要预测的小块
            imgTemp = np.zeros((block_row,block_col,3),np.uint8)
            #labelTemp = np.zeros((step_row,step_col),np.uint8)
            for i in range(0,block_row):
                for j in range(0,block_col):
                    imgTemp[i][j][0] = src_arr[y + i][x + j][0]
                    imgTemp[i][j][1] = src_arr[y + i][x + j][1]
                    imgTemp[i][j][2] = src_arr[y + i][x + j][2]
                    
                    #bigPred_R[y+i][x+j] = 255#初始化
                    #labelTemp[i][j] = label_arr[y+i][x+j]
                    
            
            #预测小块
            smallPred_R = small_predict(model, imgTemp)
            
            #合并成大块
            for i in range(step_row,block_row-step_row):
                for j in range(step_col,block_col-step_col):
                    bigPred_R[y +i][x +j] = smallPred_R[i][j]
            
            ##print count
            ##count+=1
    
    return bigPred_R


def final_predict(test_path,predict_path,label_path,model):
    block_row = 240
    block_col = 256
    
    step_row=60
    step_col=64
    
    
    ##model = AtrousFCN_Resnet50_16s(input_shape=(240, 256, 3))
    ##model.load_weights(checkpoint_path, by_name=True)
    
    srcImg = Image.open(test_path).convert("RGB")
    labelImg = Image.open(label_path)
    #labelImg = Image.open(label_path).convert("L")
    src_arr = np.array(srcImg)
    label_arr = np.array(labelImg)
    
    img_src_pad = cv2.copyMakeBorder(src_arr,60,60,64,64,cv2.BORDER_CONSTANT,value=[255,255,255])
    
    bigPred_R=big_predict(model,img_src_pad,step_row,step_col,block_row,block_col)
    
    img_big_R = draw_color(bigPred_R)
    #print("accuracy:%f")%(correctN/(src_arr.shape[0]*src_arr.shape[1]))
    
    
    roi = img_big_R[step_row:img_big_R.shape[0]-step_row, step_col:img_big_R.shape[1]-step_col]
    Image.fromarray(roi).save(predict_path) 
    
    
print "done"

Using TensorFlow backend.


done


In [1]:
import datetime
starttime = datetime.datetime.now() #计时开始

#checkpoint_path = "Models/AtrousFCN_Resnet50_16s/jilu/checkpoint_weights(-16).hdf5"
checkpoint_path = "Models/AtrousFCN_Resnet50_16s/checkpoint_weights.hdf5"

model = AtrousFCN_Resnet50_16s(input_shape=(240, 256, 3))
model.load_weights(checkpoint_path, by_name=True)

#big_predict(test_path,predict_path,label_path,checkpoint_path)
for n in range(2,52):
    
    starttime_small = datetime.datetime.now() #计时开始2
    
    test_path = "data1/origin/src-%d.jpg"%(n)
    label_path = "data1/origin/label-%d.png"%(n)
    predict_path = "data2/val/%d_(test).png"%(n)
    final_predict(test_path,predict_path,label_path,model)
    
    endtime_small = datetime.datetime.now() #计时结束
    print (endtime_small - starttime_small).seconds

print "done"

endtime = datetime.datetime.now() #计时结束
print (endtime - starttime).seconds

NameError: name 'AtrousFCN_Resnet50_16s' is not defined

In [ ]:
##2 3 6 7 10 11  14-- 15 17 18 21 22-- 25 28  29 32-- 33 36 37 40 43-- 45 47 50 

import datetime
starttime = datetime.datetime.now() #计时开始

#checkpoint_path = "Models/AtrousFCN_Resnet50_16s/jilu/checkpoint_weights(-16).hdf5"
checkpoint_path = "Models/AtrousFCN_Resnet50_16s/checkpoint_weights.hdf5"

model = AtrousFCN_Resnet50_16s(input_shape=(240, 256, 3))
model.load_weights(checkpoint_path, by_name=True)

test_tab=[14,22,32,43]
test_tab2=[10,17,21,28,36]##32

#big_predict(test_path,predict_path,label_path,checkpoint_path)
for n in range(2,52):
    if(n in test_tab2):
        starttime_small = datetime.datetime.now() #计时开始2

        test_path = "data1/origin/src-%d.jpg"%(n)
        label_path = "data1/origin/label-%d.png"%(n)
        predict_path = "data2/val2/%d_(test)_step1.png"%(n)
        final_predict(test_path,predict_path,label_path,model)

        endtime_small = datetime.datetime.now() #计时结束
        print (endtime_small - starttime_small).seconds

print "done"

endtime = datetime.datetime.now() #计时结束
print (endtime - starttime).seconds

300
276
282
271


In [2]:
##2 3 6 7 10 11  14-- 15 17 18 21 22-- 25 28  29 32-- 33 36 37 40 43-- 45 47 50 

import datetime
starttime = datetime.datetime.now() #计时开始

#checkpoint_path = "Models/AtrousFCN_Resnet50_16s/jilu/checkpoint_weights(-16).hdf5"
checkpoint_path = "Models/AtrousFCN_Resnet50_16s/checkpoint_weights_hd-17.hdf5"

model = AtrousFCN_Resnet50_16s(input_shape=(240, 256, 3))
model.load_weights(checkpoint_path, by_name=True)

test_tab2=[10,17,21,28,36]##32

n=17

#big_predict(test_path,predict_path,label_path,checkpoint_path)
starttime_small = datetime.datetime.now() #计时开始2

test_path = "data1/origin/src-%d.jpg"%(n)
label_path = "data1/origin/label-%d.png"%(n)
predict_path = "data2/val3/%d_(test)_step1.png"%(n)
final_predict(test_path,predict_path,label_path,model)

endtime_small = datetime.datetime.now() #计时结束
print (endtime_small - starttime_small).seconds

print "done"



110
done
